## Note: This code is taken from IBM's Qiskit Tutorial, Entangled States: 

https://learn.qiskit.org/course/introduction/entangled-states

### Entangling States

In [2]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

qc = QuantumCircuit(2)

# This calculates what the state vector of our qubits would be
# after passing through the circuit 'qc'

ket = Statevector(qc)
ket.draw()


'Statevector([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],\n            dims=(2, 2))'

### Now add a Hadamard Gate

In [3]:
qc.h(1)

ket = Statevector(qc)
ket.draw()

#Result gives you (√2/2)|00> + (√2/2)|10>

'Statevector([0.70710678+0.j, 0.        +0.j, 0.70710678+0.j,\n             0.        +0.j],\n            dims=(2, 2))'

### Now add a CX Gate to Create Entangled State

In [4]:
qc.cx(1,0)

ket = Statevector(qc)
ket.draw()

#This result generates an entangled state
# (√2/2)|00> + (√2/2)|11>; which can not be
# generated as a product state of two natural
# qubits

'Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,\n             0.70710678+0.j],\n            dims=(2, 2))'

### Exploring Phase Kickback with CX and Z gates

In [5]:
qc = QuantumCircuit(2)

#Hadamard Gate on qubit 0 and 1
qc.h(0)
qc.h(1)

ket = Statevector(qc)
ket.draw()

'Statevector([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j],\n            dims=(2, 2))'

In [6]:
#We see cx currently doesn't impact the vector
qc.cx(1,0)

ket = Statevector(qc)
ket.draw()

'Statevector([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j],\n            dims=(2, 2))'

In [7]:
#Flip target bit from |+> to |-> state
# using z gate
qc.z(0)

ket = Statevector(qc)
ket.draw()

'Statevector([ 0.5+0.j, -0.5+0.j,  0.5+0.j, -0.5+0.j],\n            dims=(2, 2))'

In [8]:
#Now if we use cx gate, it flips the control bit
# to |-> state as well

qc.cx(1,0)
ket = Statevector(qc)
ket.draw()

#This phenom is called "phase kickback" where
# if both control and target are in a superposition
# some features of target superposition can affect
# control

'Statevector([ 0.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j],\n            dims=(2, 2))'

## Using Entanglement for Quantum Communication

In [12]:
# The message
MESSAGE = '00'

# Alice encodes the message
qc_alice = QuantumCircuit(2,2)

if MESSAGE[-1]=='1':
    qc_alice.x(0)
if MESSAGE[-2]=='1':
    qc_alice.x(1)


In [13]:
from qiskit import Aer
backend = Aer.get_backend('aer_simulator')

# Bob measures
qc_bob = QuantumCircuit(2,2)
qc_bob.measure([0,1],[0,1])

#This run utilizes no entanglement and requires
# 2 qubits/bits from both Alice and Bob
backend.run(qc_alice.compose(qc_bob)).result().get_counts()

{'01': 1024}

### Now implement entanglement between Alice and Bob

In [16]:
# The message
MESSAGE = '00'

# Alice encodes the message
qc_alice = QuantumCircuit(2,2)

if MESSAGE[-1]=='1':
    qc_alice.x(0)
if MESSAGE[-2]=='1':
    qc_alice.x(1)

#Now create entangled states
qc_alice.h(1)
qc_alice.cx(1,0)

ket = Statevector(qc_alice)
ket.draw()

'Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,\n             0.70710678+0.j],\n            dims=(2, 2))'

In [17]:
#Now bob receives the states
# and has to disentangle
qc_bob = QuantumCircuit(2,2)

#Bob disentangles
qc_bob.cx(1,0)
qc_bob.h(1)

#Then measure
qc_bob.measure([0,1],[0,1])

qc_bob.draw()

┌───┐     ┌─┐   
q_0: ┤ X ├─────┤M├───
     └─┬─┘┌───┐└╥┘┌─┐
q_1: ──■──┤ H ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1

### To change between the four entangled states, all that is required is either a z gate or x gate. Utilizing this info, we can modify the way Alice encodes her message

In [18]:
# The message
MESSAGE = '00'

# Alice encodes the message
qc_alice = QuantumCircuit(2,2)

#Create entangled states first
qc_alice.h(1)
qc_alice.cx(1,0)

if MESSAGE[-2]=='1':
    qc_alice.z(1)
if MESSAGE[-1]=='1':
    qc_alice.x(1)

ket = Statevector(qc_alice)
#This result should match the previous version of Alice's
# encoding, where previously she was using two x gates on 
# both qubits
ket.draw()

'Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,\n             0.70710678+0.j],\n            dims=(2, 2))'

In [19]:
#Bob's circuit doesn't need to change
backend.run(qc_alice.compose(qc_bob)).result().get_counts()

{'00': 1024}

### What is interesting is that x and z gates can be applied to just a single qubit (1 in this case). Alice could theoretically qubit 0 over as soon as the entanglement between qubit 0 and 1 occurs. She could even send qubit 0, before establishing what her message would be!

### Here, we will make things more interesting by adding a third party (Charlie) whose only job is to create an entangled state and send the qubits out

In [20]:
qc_charlie = QuantumCircuit(2,2)

qc_charlie.h(1)
qc_charlie.cx(1,0)

qc_charlie.draw()

┌───┐
q_0: ─────┤ X ├
     ┌───┐└─┬─┘
q_1: ┤ H ├──■──
     └───┘     
c: 2/══════════

### Charlie can directly send one qubit to Bob, and the other to Alice. Alice can encode her two qubit message by manipulating just this single qubit, and send only this one qubit to Bob

In [21]:
# The message
MESSAGE = '01'

# Alice encodes the message
qc_alice = QuantumCircuit(2,2)

if MESSAGE[-2]=='1':
    qc_alice.z(1)
if MESSAGE[-1]=='1':
    qc_alice.x(1)


In [22]:
complete_qc = qc_charlie.compose(qc_alice.compose(qc_bob))
backend.run(complete_qc).result().get_counts()

{'01': 1024}